#### Val 

In [80]:
import glob
import pandas as pd
import json

dfs=[]
filenames = glob.glob(f"../output/parser/*.csv")

for filename in filenames:
    #filenames = glob.glob(f"../output/parser/*.csv")
    #print(filename)
    df = pd.read_csv(filename)
    project = filename.split("/")[-1].split("_")[0]
    df['project'] = project
    dfs.append(df)

df = pd.concat(dfs, ignore_index=True)

In [81]:
df.head(5)

,file,function,func_body,str_uncaught_exceptions,n_try_except,n_try_pass,n_finally,n_generic_except,n_raise,n_captures_broad_raise,...,n_captures_misplaced_bare_raise,n_try_else,n_try_return,str_except_identifiers,str_raise_identifiers,str_except_block,n_nested_try,n_bare_except,n_bare_raise_finally,project
0,/home/r4ph/desenv/phd/exception-miner/projects...,predict_seq2seq,"def predict_seq2seq(net, src_sentence, src_voc...",NaN,0,0,0,0,0,0,...,0,0,0,NaN,NaN,NaN,0,0,0,d2l-en
1,/home/r4ph/desenv/phd/exception-miner/projects...,train_seq2seq,"def train_seq2seq(net, data_iter, lr, num_epoc...",NaN,0,0,0,0,0,0,...,0,0,0,NaN,NaN,NaN,0,0,0,d2l-en
2,/home/r4ph/desenv/phd/exception-miner/projects...,forward,"def forward(self, pred, label, valid_len):\n ...",NaN,0,0,0,0,0,0,...,0,0,0,NaN,NaN,NaN,0,0,0,d2l-en
3,/home/r4ph/desenv/phd/exception-miner/projects...,load_data_nmt,"def load_data_nmt(batch_size, num_steps, num_e...",NaN,0,0,0,0,0,0,...,0,0,0,NaN,NaN,NaN,0,0,0,d2l-en
4,/home/r4ph/desenv/phd/exception-miner/projects...,build_array_nmt,"def build_array_nmt(lines, vocab, num_steps):\...",NaN,0,0,0,0,0,0,...,0,0,0,NaN,NaN,NaN,0,0,0,d2l-en


In [82]:
df.shape

(1295543, 21)

In [83]:
import numpy as np
#Couting uncaught exceptions:
#df[df.str_uncaught_exceptions.notna()].str_uncaught_exceptions.str.split("[.][.][.]").values[3]

df['n_uncaught_exceptions'] = df['str_uncaught_exceptions'].str.split("[.][.][.]").apply(lambda x: len(x)-1 if x is not np.nan else 0)

In [84]:
cols = ['n_captures_broad_raise', 'n_generic_except', 'n_try_pass', 'n_nested_try', 'n_captures_misplaced_bare_raise', 'n_bare_except', 'n_uncaught_exceptions', 'n_bare_raise_finally']

for col in cols:
    df[col+'_uni'] = df[col].apply(lambda x: 1 if x >= 1 else x)

In [85]:
df['sum_ap'] = df.sum(axis=1, numeric_only=True)

cols = ['project', 'file', 'function', 'func_body', 
        'n_captures_broad_raise_uni', 'n_generic_except_uni', 'n_try_pass_uni', 'n_nested_try_uni', 'n_captures_misplaced_bare_raise_uni', 'n_bare_except_uni', 'n_uncaught_exceptions_uni', 
        'n_bare_raise_finally_uni', 'sum_ap']

In [86]:
df_sample = df.loc[:, cols]

In [87]:
df_sample[df_sample['sum_ap'] >= 1].shape

(38694, 13)

In [88]:
df_sample = df_sample[df_sample['sum_ap'] >= 1]

Explode the Columns

In [89]:
cols =  ['n_captures_broad_raise_uni', 'n_generic_except_uni', 'n_try_pass_uni', 'n_nested_try_uni', 'n_captures_misplaced_bare_raise_uni', 'n_bare_except_uni', 'n_uncaught_exceptions_uni', 
        'n_bare_raise_finally_uni']

# Create an empty DataFrame to store the exploded data
exploded_df = pd.DataFrame(columns=['project', 'file', 'function', 'func_body', 'anti_pattern', 'count'])

# Iterate over the DataFrame and create new rows for non-zero values in specified columns
exploded_dfs = []

for index, row in df_sample.iterrows():
    for col in cols:
        if row[col] > 0:
            exploded_df = pd.DataFrame([{'project': row['project'], 'file': row['file'], 'function': row['function'], 'func_body': row['func_body'] ,'anti_pattern': col, 'count': row[col]}])
            exploded_dfs.append(exploded_df)

df_val = pd.concat(exploded_dfs, ignore_index=True)

In [90]:
df_val

,project,file,function,func_body,anti_pattern,count
0,d2l-en,/home/r4ph/desenv/phd/exception-miner/projects...,show_images,"def show_images(imgs, num_rows, num_cols, titl...",n_try_pass_uni,1
1,d2l-en,/home/r4ph/desenv/phd/exception-miner/projects...,show_images,"def show_images(imgs, num_rows, num_cols, titl...",n_bare_except_uni,1
2,d2l-en,/home/r4ph/desenv/phd/exception-miner/projects...,show_images,"def show_images(imgs, num_rows, num_cols, titl...",n_try_pass_uni,1
3,d2l-en,/home/r4ph/desenv/phd/exception-miner/projects...,show_images,"def show_images(imgs, num_rows, num_cols, titl...",n_bare_except_uni,1
4,d2l-en,/home/r4ph/desenv/phd/exception-miner/projects...,show_images,"def show_images(imgs, num_rows, num_cols, titl...",n_try_pass_uni,1
...,...,...,...,...,...,...
44301,wttr.in,/home/r4ph/desenv/exception-miner/projects/py/...,_parse_accept_language,def _parse_accept_language(accept_language):\n...,n_try_pass_uni,1
44302,wttr.in,/home/r4ph/desenv/exception-miner/projects/py/...,_parse_language_header,"def _parse_language_header(header):\n """"""\n...",n_try_pass_uni,1
44303,wttr.in,/home/r4ph/desenv/exception-miner/projects/py/...,query_osm,def query_osm(location_string):\n try:\n ...,n_generic_except_uni,1
44304,wttr.in,/home/r4ph/desenv/exception-miner/projects/py/...,load_cache,def load_cache(location_string):\n try:\n ...,n_bare_except_uni,1


In [91]:
# import numpy as np
# # Get a sample from a dataframe and split in n parts:
# shuffled = df.sample(frac=1, random_state=40)
# result = np.array_split(shuffled, 2)

In [92]:
import pandas as pd

def stratified_sample(df, target_column, sample_size, random_state=42):
    # Group by the target column and sample
    stratified_sample = df.groupby(target_column, group_keys=False, sort=False).apply(lambda x: x.sample(min(len(x), sample_size), random_state=random_state))
    return stratified_sample

# Create the stratified sample
stratified_sample_df = stratified_sample(df_val, target_column='anti_pattern', sample_size=382)

In [93]:
# Display the stratified sample
stratified_sample_df

,project,file,function,func_body,anti_pattern,count
18102,pysnooper,/home/r4ph/desenv/exception-miner/projects/py/...,main,def main():\n try:\n bar()\n exce...,n_try_pass_uni,1
33288,parlai,/home/r4ph/desenv/phd/exception-miner/projects...,perturb,"def perturb(self, example_dict: Dict) -> Dict:...",n_try_pass_uni,1
3284,wagtail,/home/r4ph/desenv/phd/exception-miner/projects...,with_content_json,"def with_content_json(self, content):\n ...",n_try_pass_uni,1
163,urh,/home/r4ph/desenv/phd/exception-miner/projects...,find,def find(self):\n result = list()\n ...,n_try_pass_uni,1
24037,pupy,/home/r4ph/desenv/phd/exception-miner/projects...,dumpCachedHashes,def dumpCachedHashes(self):\n if self._...,n_try_pass_uni,1
...,...,...,...,...,...,...
12198,django,/home/r4ph/desenv/exception-miner/projects/py/...,run_tests,"def run_tests(self, test_labels, **kwargs):\n ...",n_bare_raise_finally_uni,1
42811,edgedb,/home/r4ph/desenv/phd/exception-miner/projects...,write_data_cache,"def write_data_cache(\n obj: Any,\n cach...",n_bare_raise_finally_uni,1
39048,jina,/home/r4ph/desenv/phd/exception-miner/projects...,_get_results,"async def _get_results(\n self,\n ...",n_bare_raise_finally_uni,1
30000,sanic,/home/r4ph/desenv/exception-miner/projects/py/...,bind_unix_socket,"def bind_unix_socket(path: str, *, mode=0o666,...",n_bare_raise_finally_uni,1


In [97]:
df_val.groupby('anti_pattern').count()

,project,file,function,func_body,count
anti_pattern,,,,,
n_bare_except_uni,4189,4189,4189,4189,4189
n_bare_raise_finally_uni,10,10,10,10,10
n_captures_broad_raise_uni,5744,5744,5744,5744,5744
n_captures_misplaced_bare_raise_uni,124,124,124,124,124
n_generic_except_uni,15504,15504,15504,15504,15504
n_nested_try_uni,745,745,745,745,745
n_try_pass_uni,10219,10219,10219,10219,10219
n_uncaught_exceptions_uni,7771,7771,7771,7771,7771


In [119]:
stratified_sample_1  = df_val[df_val['anti_pattern'] != 'n_bare_raise_finally_uni'].groupby('anti_pattern').apply(lambda x: x.sample(n=53, replace=True, random_state=40)).reset_index(drop=True)

In [120]:
stratified_sample_1

,project,file,function,func_body,anti_pattern,count
0,ipython,/home/r4ph/desenv/exception-miner/projects/py/...,_run_cmd_line_code,"def _run_cmd_line_code(self):\n """"""Run ...",n_bare_except_uni,1
1,pupy,/home/r4ph/desenv/phd/exception-miner/projects...,__init__,"def __init__(self, pstore_dir='~'):\n t...",n_bare_except_uni,1
2,paramiko,/home/r4ph/desenv/phd/exception-miner/projects...,read_file,"def read_file(self, filename):\n """"""\n ...",n_bare_except_uni,1
3,impacket,/home/r4ph/desenv/phd/exception-miner/projects...,Authenticate,def Authenticate(self):\r\n packet = SM...,n_bare_except_uni,1
4,quantaxis,/home/r4ph/desenv/phd/exception-miner/projects...,__saving_work,"def __saving_work(code, coll):\n\n QA_u...",n_bare_except_uni,1
...,...,...,...,...,...,...
366,frida,/home/r4ph/desenv/phd/exception-miner/projects...,generate_devkit,"def generate_devkit(kit, host, flavor, output_...",n_uncaught_exceptions_uni,1
367,tvm,/home/r4ph/desenv/phd/exception-miner/projects...,dense_mkl,"def dense_mkl(cfg, data, weight, bias=None, ou...",n_uncaught_exceptions_uni,1
368,insightface,/home/r4ph/desenv/phd/exception-miner/projects...,iresnet18,"def iresnet18(pretrained=False, progress=True,...",n_uncaught_exceptions_uni,1
369,scipy,/home/r4ph/desenv/phd/exception-miner/projects...,cpu_count,def cpu_count(only_physical_cores=False):\n ...,n_uncaught_exceptions_uni,1


In [121]:
stratified_sample_2 = df_val[df_val['anti_pattern'] == 'n_bare_raise_finally_uni']

In [122]:
stratified_final = pd.concat([stratified_sample_1, stratified_sample_2])

In [123]:
stratified_final

,project,file,function,func_body,anti_pattern,count
0,ipython,/home/r4ph/desenv/exception-miner/projects/py/...,_run_cmd_line_code,"def _run_cmd_line_code(self):\n """"""Run ...",n_bare_except_uni,1
1,pupy,/home/r4ph/desenv/phd/exception-miner/projects...,__init__,"def __init__(self, pstore_dir='~'):\n t...",n_bare_except_uni,1
2,paramiko,/home/r4ph/desenv/phd/exception-miner/projects...,read_file,"def read_file(self, filename):\n """"""\n ...",n_bare_except_uni,1
3,impacket,/home/r4ph/desenv/phd/exception-miner/projects...,Authenticate,def Authenticate(self):\r\n packet = SM...,n_bare_except_uni,1
4,quantaxis,/home/r4ph/desenv/phd/exception-miner/projects...,__saving_work,"def __saving_work(code, coll):\n\n QA_u...",n_bare_except_uni,1
...,...,...,...,...,...,...
41505,memray,/home/r4ph/desenv/phd/exception-miner/projects...,run_till_snapshot_point,def run_till_snapshot_point(\n program: str...,n_bare_raise_finally_uni,1
42101,celery,/home/r4ph/desenv/exception-miner/projects/py/...,trace_task,"def trace_task(\n uuid: str,\n ...",n_bare_raise_finally_uni,1
42107,celery,/home/r4ph/desenv/exception-miner/projects/py/...,build_tracer,"def build_tracer(\n name: str,\n ...",n_bare_raise_finally_uni,1
42795,edgedb,/home/r4ph/desenv/phd/exception-miner/projects...,__aenter__,"async def __aenter__(self):\n status_r,...",n_bare_raise_finally_uni,1


In [128]:
stratified_final.groupby('anti_pattern').sum('count').reset_index()

,anti_pattern,count
0,n_bare_except_uni,53
1,n_bare_raise_finally_uni,10
2,n_captures_broad_raise_uni,53
3,n_captures_misplaced_bare_raise_uni,53
4,n_generic_except_uni,53
5,n_nested_try_uni,53
6,n_try_pass_uni,53
7,n_uncaught_exceptions_uni,53
